<a href="https://colab.research.google.com/github/aTunass/segment/blob/main/Segment_self.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchmetrics
!pip install segmentation_models_pytorch
!pip install albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=ac7bb353838c251ab6072cfadc508e48131274a3bb5f790d22dce0911d378df0
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1

In [ ]:
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz

--2023-06-17 14:17:48--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/datasets/pets/images.tar.gz [following]
--2023-06-17 14:17:49--  https://thor.robots.ox.ac.uk/datasets/pets/images.tar.gz
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  15.4MB/s    in 42s     

2023-06-17 14:18:31 (17.9 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2023-06-17 14:18:31--  https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.

In [ ]:
#giải nén file
!tar -xf annotations.tar.gz
!tar -xf images.tar.gz

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchmetrics
from torchmetrics import Dice, JaccardIndex #(IOU???)
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2 # np.array -> torch.tensor
import os
from tqdm import tqdm
from glob import glob

In [ ]:
mask=cv2.imread("/content/annotations/trimaps/Abyssinian_100.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(mask)
print(mask)
print(mask.shape)
print(np.unique(mask)) #những loại pixel

In [ ]:
image = cv2.imread("/content/images/Abyssinian_100.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
print(image.shape)

In [ ]:
img_path_list = []
with open("/content/annotations/trainval.txt") as file_in:
    for line in file_in:
      img_path_list.append(line.split(" ")[0])
print(img_path_list)

FileNotFoundError: ignored

In [ ]:
image_path = os.path.join("/content", "images", "{}.jpg".format(img_path_list[17]))
print(image_path)

/content/images/Abyssinian_116.jpg


In [ ]:
mask_path = os.path.join("/content", "annotations", "trimaps", "{}.png".format(img_path_list[17]))
print(mask_path)

/content/annotations/trimaps/Abyssinian_116.png


In [ ]:
class my_dataset(Dataset):
  def __init__(self, root_dir, txt_file, transform=None): #transform: agumentation + norm ([0-255] -> [0-1]) + np.array->tensor
        super().__init__()
        self.root_dir = root_dir
        self.txt_file = txt_file
        self.transform = transform
        self.img_path_list = []
        with open(self.txt_file) as file_in:
          for line in file_in:
            self.img_path_list.append(line.split(" ")[0])
  def __len__(self):
      return len(self.img_path_list)

  def __getitem__(self, idx):
      image_path = os.path.join(self.root_dir, "images", "{}.jpg".format(self.img_path_list[idx]))
      mask_path = os.path.join(self.root_dir, "annotations", "trimaps", "{}.png".format(self.img_path_list[idx]))
      image = cv2.imread(image_path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      # foreground -> 1 chuyển các pixel về 0 và 1 để dễ phân loại và thỏa mãn yêu cầu class của pytorch phải từ 0 đến
      # background 2 -> 0
      # 3 -> 1
      mask[mask == 2] = 0
      mask[mask == 3] = 1
      if self.transform is not None:
          transformed = self.transform(image=image, mask=mask)
          transformed_image = transformed['image']
          transformed_mask = transformed['mask']
      return transformed_image, transformed_mask

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
#accuracy fn
def accuracy_function(preds, targets):
    preds_flat = preds.flatten()
    targets_flat = targets.flatten()
    acc = torch.sum(preds_flat == targets_flat)
    return acc/targets_flat.shape[0]

In [ ]:
trainsize = 384
train_transform = A.Compose([
    A.Resize(width=trainsize, height=trainsize),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Blur(),
    A.Sharpen(),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, always_apply=False, p=0.5),
    A.Cutout(num_holes=5, max_h_size=25, max_w_size=25, fill_value=0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0), #mean và std là của tập imageNet, vì dử dụng pretrained model trên tập này
    ToTensorV2(), # đưa về tensor
])
test_transform =  A.Compose([
    A.Resize(width=trainsize, height=trainsize),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
    ToTensorV2(), # numpy.array -> torch.tensor (B, 3, H, W)
])

NameError: ignored

In [ ]:
train_dataset = my_dataset("/content", "/content/annotations/trainval.txt", train_transform)
test_dataset = my_dataset("/content", "/content/annotations/test.txt", test_transform)
image, mask = test_dataset.__getitem__(17)
print(image.shape, mask.shape)
print(mask.unique())
plt.subplot(1, 2, 1)
plt.imshow(unorm(image).permute(1, 2, 0)) #channel đang nằm đầu phải chuyển về cuối mới hiển thị ảnh được
plt.subplot(1, 2, 2)
plt.imshow(mask)
plt.show()

In [ ]:
##model Unet
def Unet_block(in_channels, out_channels):
  return nn.Sequential(
      nn.Conv2d(in_channels, out_channels, 3, 1, 1),
      nn.ReLU(),
      nn.Conv2d(out_channels, out_channels, 3, 1, 1),
      nn.ReLU(),
  )
class Unet(nn.Module):
  def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.downsample = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear")
        self.block_down1 = Unet_block(3,64)
        self.block_down2 = Unet_block(64,128)
        self.block_down3 = Unet_block(128,256)
        self.block_down4 = Unet_block(256,512)
        self.block_neck = Unet_block(512,1024)
        self.block_up1 = Unet_block(1024+512,512) #upsample tu neck ra channel:512 + channel tu down4: 1024 qua concat
        self.block_up2 = Unet_block(256+512,256)
        self.block_up3 = Unet_block(128+256,128)
        self.block_up4 = Unet_block(64+128,64)

        self.block_conv = nn.Conv2d(64,self.n_classes, 1) #ket qua: tensor (batch_size, n_classes, H, W)

  def forward(self, x):
     x1 = self.block_down1(x)
     x = self.downsample(x1)
     x2 = self.block_down2(x)
     x = self.downsample(x2)
     x3 = self.block_down3(x)
     x = self.downsample(x3)
     x4 = self.block_down4(x)
     x = self.downsample(x4)

     x = self.block_neck(x)

     x = torch.cat([x4, self.upsample(x)], dim=1)# concat theo chieu channel
     x = self.block_up1(x)
     x = torch.cat([x3, self.upsample(x)], dim=1)
     x = self.block_up2(x)
     x = torch.cat([x2, self.upsample(x)], dim=1)
     x = self.block_up3(x)
     x = torch.cat([x1, self.upsample(x)], dim=1)
     x = self.block_up4(x)

     x = self.block_conv(x)
     return x
# model = Unet(1)
# x = torch.rand(4,3, trainsize, trainsize)
# print(x.shape)
# y = model(x)
# print(y.shape)
# print(y.squeeze().shape)

In [ ]:
!nvidia-smi

In [ ]:
#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_dataset = my_dataset("/content", "/content/annotations/trainval.txt", train_transform)
test_dataset = my_dataset("/content", "/content/annotations/test.txt", test_transform)
from torch.utils.data import DataLoader
#load data
batch_size = 8
n_worker = os.cpu_count()
print("num_worker", n_worker)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=n_worker)

#model
model = Unet(1).to(device)
#loss_training
criterion = nn.BCEWithLogitsLoss()
#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
n_eps = 10
#metrics tính toán độ chính xác
dice_fn = torchmetrics.Dice(num_classes=2, average="macro").to(device)
iou_fn = torchmetrics.JaccardIndex(num_classes=2, task="binary", average="macro").to(device)
#meter
acc_meter = AverageMeter()
train_loss_meter = AverageMeter()
dice_meter = AverageMeter()
iou_meter = AverageMeter()

In [ ]:
#training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/UNet/model_ep_17.pth", map_location=device))
for ep in range(1,1+n_eps):
  acc_meter.reset()
  train_loss_meter.reset()
  dice_meter.reset()
  iou_meter.reset()
  model.train() #chuyển sang chế độ training

  for batch_id, (x,y) in enumerate(tqdm(train_dataloader), start=1):
    optimizer.zero_grad()
    n = x.shape[0] #bacth_size
    x = x.to(device).float() #image
    y = y.to(device).float() #mask
    y_hat = model(x) #predict # -> logit (-vc, +vc) giá trị từ âm vc đến dương vc
    # y_true: (batch_size ,384, 384), y_hat: (batch_size, n_classses ,384, 384), chuyển h_hat về cùng dạng y_true
    y_hat = y_hat.squeeze()
    loss = criterion(y_hat, y)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      y_hat_mask = y_hat.sigmoid().round().long() # -> mask (0, 1)
      dice_score = dice_fn(y_hat_mask, y.long())
      iou_score = iou_fn(y_hat_mask, y.long())
      accuracy = accuracy_function(y_hat_mask, y.long())
      train_loss_meter.update(loss.item(), n)
      iou_meter.update(iou_score.item(), n)
      dice_meter.update(dice_score.item(), n)
      acc_meter.update(accuracy.item(), n)
  print("EP {}, train loss = {}, accuracy = {}, IoU = {}, dice = {}".format(
        ep, train_loss_meter.avg, acc_meter.avg, iou_meter.avg, dice_meter.avg
    ))
  if ep >= 5:
        torch.save(model.state_dict(), "/content/drive/MyDrive/UNet/model_ep_{}.pth".format(ep))


In [ ]:
# đưa ra kết quả trên tập dữ liệu test
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/UNet/model_ep_17.pth", map_location=device))
model.eval()
test_dice_meter = AverageMeter()
test_iou_meter = AverageMeter()
with torch.no_grad():
  for batch_id, (x, y) in enumerate(tqdm(test_dataloader), start=1):
        n = x.shape[0]
        x = x.to(device).float()
        y = y.to(device).float()
        y_hat = model(x) #giá trị logit chưa qua sigmoid
        y_hat = y_hat.squeeze()
        y_hat_mask = y_hat.sigmoid().round().long()
        dice_score = dice_fn(y_hat_mask, y.long())
        iou_score = iou_fn(y_hat_mask, y.long())
        test_dice_meter.update(dice_score.item(), n)
        test_iou_meter.update(iou_score.item(), n)
print("TEST: IoU = {}, dice = {}".format(test_iou_meter.avg, test_dice_meter.avg))

In [ ]:
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/UNet/model_ep_10.pth", map_location=device))
model.eval()
idx = random.randint(0, 100)
with torch.no_grad():
    x, y = test_dataset[20]
    # print(x.shape, y.shape) (C, H, W) -> (1, C, H, W) -> model
    x = x.to(device).float().unsqueeze(0)
    y_hat = model(x).squeeze() #(1, 1, H, W) -> (H, W)
    y_hat_mask = y_hat.sigmoid().round().long()
    # x, y, y_hat_mask
    plt.subplot(1, 3, 1)
    plt.imshow(unorm(x.squeeze().cpu()).permute(1, 2, 0)) # x (GPU) -> x(CPU)
    plt.subplot(1, 3, 2)
    plt.imshow(y)
    plt.subplot(1, 3, 3)
    plt.imshow(y_hat_mask.cpu())

In [ ]:
txt_file = "/content/annotations/trainval.txt"
my_list = []
with open(txt_file) as file_in:
            for line in file_in:
                my_list.append(line.split(" ")[0])
print(my_list)